In [91]:
import os
import gzip
import shutil
import json
import pandas as pd
import _pickle as pickle
import numpy as np

## Data Preprocessing -1
Aggregation to each person 

In [2]:
df_file = pd.DataFrame()

In [3]:
def transfer_dataframe(patient_id, file_name):
    with open(file_name) as file:
        j_object = json.load(file)
        df = pd.DataFrame(j_object)
#         try:
#             df['datetime'] = pd.Timestamp(df['dateString']).to_datetime()
        df['datetime'] = pd.to_datetime(df['dateString'], utc=True)
#         except ValueError:
# #             df['datetime'] = pd.to_datetime(df['dateString'],format="%d/%m/%Y %H:%M:%S %p")
#         else:
#             raise
        
        df['pid'] = patient_id
        df_result = df[['pid', 'datetime', 'sgv']]
        df_result = df_result.sort_values(by=['datetime'], ascending = True)
    return df_result

In [ ]:
count = 0
for folder in os.listdir(os.getcwd()+'/OpenAPS'):
    print('PID:', folder)
    record_path = 'OpenAPS/'+folder+'/direct-sharing-31'
    if os.path.exists(record_path):
        files = os.listdir(record_path)
        df_folder = pd.DataFrame([], columns=['pid', 'datetime', 'sgv'])
        for file in files:
            if file.find('entries') > -1 and file.endswith('.json'):
                try:
                    df = transfer_dataframe(folder, record_path+'/'+file)
                except :
                    print(folder, file, 'cannot be transfered to datetime, pass' )
                    continue
                  
                df_folder = df_folder.append(df)
        print('before drop', len(df_folder))
        df_folder.drop_duplicates('datetime',inplace = True)
        print('after drop', len(df_folder))
        print('saving result...')
        pickle.dump(df_folder, open(os.getcwd()+'/OpenAPS/entry_dfs/'+folder+'_entries.pkl', 'wb'))
        print('done for', folder)

        

In [58]:
try:
    pd.to
pd.to_datetime('10/09/2016 19:39:21 PM',format="%d/%m/%Y %H:%M:%S %p")

Timestamp('2016-09-10 19:39:21')

In [64]:
pd.to_datetime('2017-10-08T19:58:22.681-0400', utc=True)

Timestamp('2017-10-08 23:58:22.681000+0000', tz='UTC')

In [5]:
len(os.listdir(os.getcwd()+'/OpenAPS/entry_dfs'))
    

110

In [ ]:
path = os.getcwd()+'/OpenAPS/entry_dfs'
for file in os.listdir(path):
    df = pickle.load(open(path+'/'+file, 'rb'))
    print(df.head())

## Data preprocessing -2
Find same time intervals

In [ ]:
count = 0
for file in os.listdir(path):
    df = pickle.load(open(path+'/'+file, 'rb'))
#     mask = (df['datetime'] > '2017-03-01') & (df['datetime'] < '2017-03-11')
    df.set_index('datetime',inplace = True)
#     df.index = df.tz_convert('America/Los_Angeles')
    df = df.tz_convert(None)

    df_selected = df['2017-03-01':'2017-03-11']
    if len(df_selected)>0:
        print(df_selected.index[:2])
        print(df_selected['pid'].values[0])
        pickle.dump(df_selected, open(os.getcwd()+'/OpenAPS/entry_dfs_sliced/'+df_selected['pid'].values[0]+'.pkl', 'wb'))
        count += 1
print('Instances found:', count)        

In [ ]:
path2 = os.getcwd()+'/OpenAPS/entry_dfs_sliced/'

files = os.listdir(path2)

df1 = pickle.load(open(path2+'/'+files[0], 'rb'))
df2 = pickle.load(open(path2+'/'+files[1], 'rb'))

## Compute distance with Dynamic Time Wrappinng

In [51]:
import math
def DTWDistance(s1, s2,w):
    DTW={}
    
    w = max(w, abs(len(s1)-len(s2)))
    
    for i in range(-1,len(s1)):
        for j in range(-1,len(s2)):
            DTW[(i, j)] = float('inf')
    DTW[(-1, -1)] = 0
  
    for i in range(len(s1)):
        for j in range(max(0, i-w), min(len(s2), i+w)):
            dist= (s1[i]-s2[j])**2
            DTW[(i, j)] = dist + min(DTW[(i-1, j)],DTW[(i, j-1)], DTW[(i-1, j-1)])
            
    return math.sqrt(DTW[len(s1)-1, len(s2)-1])

In [136]:
def LB_Keogh(s1,s2,r):
    LB_sum=0
    for ind,i in enumerate(s1):
        
        lower_bound=min(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
        upper_bound=max(s2[(ind-r if ind-r>=0 else 0):(ind+r)])
#         print('i', i)
#         print('upper_bound', upper_bound)
        if i>upper_bound:
            LB_sum=LB_sum+(i-upper_bound)**2
        elif i<lower_bound:
            LB_sum=LB_sum+(i-lower_bound)**2
    
    return math.sqrt(LB_sum)

In [67]:
DTWDistance(df1['sgv'].values[:288], df2['sgv'].values[:288], 96)

365.276607518193

In [71]:
LB_Keogh(df1['sgv'].values[:288], df2['sgv'].values[:288], 96)

91.03296106356203

## Clustering with KNN

In [ ]:
# convert dataframe into dicts
def transfer_dicts(path, df_files):
    rst = {}
    for file in df_files:
        df = pickle.load(open(path + file, 'rb'))        
        
    return rst

In [145]:
rst = {}
for file in os.listdir(path):
    df = pickle.load(open(path+'/'+file, 'rb'))
    df.set_index('datetime',inplace = True)
    df = df.tz_convert(None)
    df.dropna(inplace=True)
    if len(df)==0:
        continue
    pid =np.asscalar(df['pid'].values[:1])
#     print('pid', pid)
    bgs = df.sgv.values[:288]
#     print('len of bgs', len(bgs))
    if len(bgs)>0:
        rst[int(pid)] = bgs

print(len(rst))        

79


In [146]:
import random

def k_means_clust(bg_dicts,num_clust,num_iter,w=5):
    keys = list(bg_dicts.keys())
    # for the final result backtracking
    search_dict = {}
    for i in range(len(keys)):
        search_dict[i] = keys[i]
    data = list(bg_dicts.values())
    data = [[int(l) for l in r] for r in data]
    centroids = random.sample(data, num_clust)
       
    counter=0
    for n in range(num_iter):
        counter+=1
        print(counter)
        assignments={}
        #assign data points to clusters
        for ind,i in enumerate(data):
            min_dist=float('inf')
            closest_clust=None
            for c_ind,j in enumerate(centroids):
                lb = LB_Keogh(i,j,5)
#                 print('lb', lb)
#                 print('min_dist', min_dist)
                if lb<min_dist:
                    cur_dist=DTWDistance(i,j,w)
                    if cur_dist<min_dist:
                        min_dist=cur_dist
                        closest_clust=c_ind
#             if closest_clust in assignments:
#                 assignments[closest_clust].append(ind)
#             else:
#                 assignments[closest_clust]=[]                
            assignments.setdefault(closest_clust,[])
            assignments[closest_clust].append(ind)
    
        #recalculate centroids of clusters
        for key in assignments:
            clust_sum=np.zeros(len(data[0]))
            for k in assignments[key]:
                clust_sum=clust_sum+data[k]
                
            centroids[key]=[m/len(assignments[key]) for m in clust_sum]
    
    return centroids,assignments

In [147]:
centroids, assignnments = k_means_clust(rst, 4, 10)

1
2
3
4
5
6
7
8
9
10


In [148]:
print(assignnments)

{3: [0, 5, 10, 14, 15, 16, 17, 19, 21, 25, 26, 29, 33, 36, 45, 48, 49, 53, 56, 57, 60, 62, 67, 68, 75, 76, 77], 0: [1, 18, 20, 30, 32, 35, 37, 39, 47, 50, 52, 58, 64, 65], 1: [2, 3, 6, 8, 11, 12, 23, 24, 27, 28, 31, 34, 38, 40, 41, 42, 43, 61, 66, 70, 71, 72, 78], 2: [4, 7, 9, 13, 22, 44, 46, 51, 54, 55, 59, 63, 69, 73, 74]}


In [154]:
keys = list(rst.keys())
clusters = {}
for k,v in assignnments.items():
    clusters[keys[k]] = [keys[i] for i in v]

for k,v in clusters.items():
    print(k,v)

57176789 [27700103, 84984656, 89710417, 897741, 28768536, 1884126, 1352464, 80373992, 89727223, 13029224, 66836068, 13484299, 32407882, 17161370, 7886752, 46966807, 98974339, 74077367, 14092221, 99296581, 62401782, 24587372, 45025419, 37948668, 39986716, 87770486, 88004055]
27700103 [96254963, 3572116, 67208817, 97417885, 67359234, 5274556, 47750728, 96484928, 60844515, 37764532, 85199788, 96805916, 21946407, 71397255]
96254963 [73521474, 57176789, 71236754, 49796612, 28176124, 40237051, 80501215, 24448124, 66019205, 95851255, 20216809, 84081904, 78420229, 77104076, 94875538, 41663654, 13783771, 95614431, 72492570, 99848889, 91161972, 38110191, 64024750]
73521474 [51336122, 43104037, 20396154, 79526193, 99908129, 37875431, 84589080, 43589707, 4762925, 34148224, 40997757, 84109428, 25692073, 15634563, 80796147]
